### IMPORTANT

OpenAI's API takes too long time to complete the embedding of ontology layer

In [1]:
import sys
import os
from dotenv import load_dotenv

root_dir = os.path.abspath("..")
sys.path.append(root_dir)
dotenv_path = os.path.join(root_dir, ".env")
load_dotenv(dotenv_path)

True

In [2]:
from pathlib import Path
import math
import polars as pl
import numpy as np
from sentence_transformers import SentenceTransformer

/Users/goonzard/Developer/data-science-09-kaggle-airplane/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
PARQUET_PATH = os.path.join(root_dir, "data", "processed_flight_features_train.parquet")
OUTPUT_DIR = os.path.join(root_dir, "data", "embedded_flight_feature_lite_train")

PARQUET_OUT_DIR = os.path.join(root_dir, "data", "embedded_flight_feature_lite_parquet_train")

COL_NAME = "flight_text"
MODEL_NAME = "all-MiniLM-L6-v2"

In [4]:
scan = pl.scan_parquet(PARQUET_PATH).select([COL_NAME]).with_row_index("row_id")
row_count = scan.select(pl.len()).collect(engine="streaming")[0, 0]
print(f"[INFO] Total rows: {row_count}")

[INFO] Total rows: 18145372


In [5]:
model = SentenceTransformer(MODEL_NAME)

In [ ]:
start = 17600384  # Edit here for resuming
chunk_idx = 137502  # Edit here for resuming
BATCH_SIZE = 128

while start < row_count:
    end = min(start + BATCH_SIZE, row_count)
    print(f"[INFO] Processing rows {start} to {end} of {row_count}")

    # Collect this chunk
    df_chunk = (
        pl.scan_parquet(PARQUET_PATH)
        .select([COL_NAME])
        .with_row_index("row_id")
        .filter((pl.col("row_id") >= start) & (pl.col("row_id") < end))
        .collect(engine="streaming")
    )

    texts = df_chunk[COL_NAME].to_list()
    row_ids = df_chunk["row_id"].to_list()

    # Encode in smaller batches
    all_embeddings = []
    for i in range(0, len(texts), BATCH_SIZE):
        subtexts = texts[i:i+BATCH_SIZE]
        emb = model.encode(
            subtexts,
            batch_size=BATCH_SIZE,
            show_progress_bar=False,
            convert_to_numpy=True,
            normalize_embeddings=True
        )
        all_embeddings.append(emb)

    all_embeddings = np.vstack(all_embeddings)  # shape: (chunk_size, 384)
    # Save embeddings and row_ids
    out_file = os.path.join(OUTPUT_DIR, f"embeddings_part{chunk_idx:05d}.npz")
    np.savez_compressed(out_file, row_ids=np.array(row_ids), embeddings=all_embeddings)
    print(f"[INFO] Saved {len(row_ids)} embeddings to {out_file}")

    # Next chunk
    start = end
    chunk_idx += 1


In [36]:
# find all npz files
NUM_OUTPUTS = 15

npz_files = sorted(Path(OUTPUT_DIR).glob("embeddings_part*.npz"))
total_files = len(npz_files)
print(f"[INFO] Found {total_files} chunk files")

# how many files per parquet group (ceil)
files_per_split = math.ceil(total_files / NUM_OUTPUTS)

for split_idx in range(NUM_OUTPUTS):
    start = split_idx * files_per_split
    end = min((split_idx + 1) * files_per_split, total_files)
    split_files = npz_files[start:end]

    if not split_files:  # no files left
        break

    print(f"[INFO] Processing split {split_idx+1}/{NUM_OUTPUTS}: files {start} to {end-1} ({len(split_files)} files)")

    all_tables = []

    for f in split_files:
        data = np.load(f)
        row_ids = data["row_ids"]
        embeddings = data["embeddings"]
        n_samples, dim = embeddings.shape

        embed_cols = {f"emb_{i}": embeddings[:, i] for i in range(dim)}

        df = pl.DataFrame({
            "row_id": row_ids,
            **embed_cols
        })
        all_tables.append(df)

    merged_df = pl.concat(all_tables, how="vertical")
    merged_df = merged_df.sort("row_id")

    out_path = os.path.join(PARQUET_OUT_DIR, f"merged_part{split_idx:02d}.parquet")
    merged_df.write_parquet(out_path)
    print(f"[INFO] Saved split {split_idx+1} -> {out_path}")

[INFO] Found 141761 chunk files
[INFO] Processing split 1/15: files 0 to 9450 (9451 files)
[INFO] Saved split 1 -> /Users/goonzard/Developer/data-science-09-kaggle-airplane/data/embedded_flight_feature_lite_parquet_train/merged_part00.parquet
[INFO] Processing split 2/15: files 9451 to 18901 (9451 files)
[INFO] Saved split 2 -> /Users/goonzard/Developer/data-science-09-kaggle-airplane/data/embedded_flight_feature_lite_parquet_train/merged_part01.parquet
[INFO] Processing split 3/15: files 18902 to 28352 (9451 files)
[INFO] Saved split 3 -> /Users/goonzard/Developer/data-science-09-kaggle-airplane/data/embedded_flight_feature_lite_parquet_train/merged_part02.parquet
[INFO] Processing split 4/15: files 28353 to 37803 (9451 files)
[INFO] Saved split 4 -> /Users/goonzard/Developer/data-science-09-kaggle-airplane/data/embedded_flight_feature_lite_parquet_train/merged_part03.parquet
[INFO] Processing split 5/15: files 37804 to 47254 (9451 files)
[INFO] Saved split 5 -> /Users/goonzard/Develo

In [4]:
# Collect all parquet files (adjust pattern if needed)
parquet_files = sorted(Path(PARQUET_OUT_DIR).glob("merged_part*.parquet"))
print(f"[INFO] Found {len(parquet_files)} parquet files")

all_tables = []

for f in parquet_files:
    print(f"[INFO] Loading {f}")
    df = pl.read_parquet(f)
    all_tables.append(df)

# Merge them into one DataFrame
merged_df = pl.concat(all_tables, how="vertical")

# (Optional) sort by row_id if needed
merged_df = merged_df.sort("row_id")

print(f"[INFO] Final merged shape: {merged_df.shape}")

[INFO] Found 15 parquet files
[INFO] Loading /Users/goonzard/Developer/data-science-09-kaggle-airplane/data/embedded_flight_feature_lite_parquet_train/merged_part00.parquet
[INFO] Loading /Users/goonzard/Developer/data-science-09-kaggle-airplane/data/embedded_flight_feature_lite_parquet_train/merged_part01.parquet
[INFO] Loading /Users/goonzard/Developer/data-science-09-kaggle-airplane/data/embedded_flight_feature_lite_parquet_train/merged_part02.parquet
[INFO] Loading /Users/goonzard/Developer/data-science-09-kaggle-airplane/data/embedded_flight_feature_lite_parquet_train/merged_part03.parquet
[INFO] Loading /Users/goonzard/Developer/data-science-09-kaggle-airplane/data/embedded_flight_feature_lite_parquet_train/merged_part04.parquet
[INFO] Loading /Users/goonzard/Developer/data-science-09-kaggle-airplane/data/embedded_flight_feature_lite_parquet_train/merged_part05.parquet
[INFO] Loading /Users/goonzard/Developer/data-science-09-kaggle-airplane/data/embedded_flight_feature_lite_parqu

In [5]:
out_path = os.path.join(root_dir, "data", "embed_flight_feature_train.parquet")
merged_df.write_parquet(out_path)